In [1]:
import pandas as pd
import numpy as np
import re

I import the subset from ZINC15 taken from Chris' dataset on the rostserver: "canonical_smiles.txt". It conains 100 canonical SMILES.

In [2]:
subset = pd.read_table('../datasets/sets_from_Chris/canonical_smiles_first_100.txt', header=None)

subset = subset.drop_duplicates()          # throw out duplicates

In [3]:
subset = subset.rename(columns={0: "canonical_SMILES"})

- Wofür steht das @? \
In SMILES, tetrahedral centers may be indicated by a simplified chiral specification (@ or @@) written as an atomic property following the atomic symbol of the chiral atom.\
The symbol "@" indicates that the following neighbors are listed anticlockwise (it is a "visual mnemonic" in that the symbol looks like an anticlockwise spiral around a central circle).\
"@@" indicates that the neighbors are listed clockwise (you guessed it, anti-anti-clockwise). 


- Wurde die Transformation mit RDKit gemacht?

### Find out the alphabet

Are we interested in the non-alphabetic signs? I can't think of any.

In [4]:
total_SMILES = "".join(subset.canonical_SMILES)

There are lower-case letters because:
- The second letter of two-character symbols must be entered in lower case.
- Atoms in aromatic rings are specified by lower case letters, e.g., aliphatic carbon is represented by the capital letter C, aromatic carbon by lower case c.

=> I must find also 2-letter elements!

First, extract the 2-letter elements.

In [5]:
two_letter_elements = re.findall("[A-Z][a-z]", total_SMILES)
two_letter_elements = sorted(set(two_letter_elements))

In [6]:
total_SMILES = re.sub("[A-Z][a-z]", "", total_SMILES)

In [7]:
only_alpha = ""

for char in total_SMILES:
    
    # CHECKING FOR UPPER CASE
    if ord(char) >= 65 and ord(char) <= 90:      # ord(chr) returns the ascii value
        only_alpha += char
        
        # checking for lower case        
    elif ord(char) >= 97 and ord(char) <= 122:
        only_alpha += char

#print(only_alpha)

In the following, I want to ignore cases, as in 'C' and 'c' only count as 'C' in the final alphabet.

In [8]:
only_alpha = only_alpha.upper()
alphabet = sorted(set(only_alpha))
alphabet = sorted(alphabet + two_letter_elements)   # Combine the element lists
alphabet

['C', 'Cc', 'Cl', 'F', 'H', 'N', 'Nc', 'O', 'Oc', 'P', 'S', 'Si']

Attention: Handle non-chemical list elements, lice Oc   ( and Cc)
Occurs e.g. in subset['canonical_SMILES'][5]

In [9]:
# subset['canonical_SMILES'][5]

'COc1cc2oc(-c3ccc(O)cc3)cc(=O)c2c(O)c1[C@@H]1O[C@H](CO)[C@@H](O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)[C@H](O)[C@H]1O'

# Find the alphabet using RdKit with SMARTS syntax
Available: subset

SMARTS syntax: wildcards *

http://www.rdkit.org/docs/GettingStartedInPython.html#substructure-searching

In [14]:
subset.head()

,canonical_SMILES
0,C[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@H]5OC(=O...
1,C[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@@H]5OC(=...
2,C[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@H]5OC(=O...
3,C[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@@H]5OC(=...
4,O=c1cc(-c2ccc(O)cc2)oc2c(O)c(O[C@@H]3O[C@H](CO...


In [16]:
import rdkit.Chem as Chem

In [17]:
m = Chem.MolFromSmiles('C1=CC=CC=C1OC')
m.HasSubstructMatch(Chem.MolFromSmiles('CO'))

True

Idee: find all present substructures => "alphabet"